## Optical Character Recognition

[HMI Text OCR](https://apmonitor.com/dde/index.php/Main/TextOCR) in the [Data-Driven Engineering](http://apmonitor.com/dde) online course.

<img align=left width=500px src='https://apmonitor.com/dde/uploads/Main/image_ocr.png'>

Recognizing text information from Human Machine Interface (HMI) screens is an important interface with legacy or proprietary systems to digitize values. These systems might lack open data communication capabilities such as Modbus, MQTT, OPC UA, Websockets, or . OCR recognition of the text on the screen may be important for one or multiple of these reasons:

- **Data Accessibility:** Legacy and proprietary systems may not provide easy access to the data in digital formats. Extracting text information from HMI screens allows for digitization, making it accessible for further analysis and integration with modern systems.
- **Automation:** Manual data entry from HMI screens is time-consuming and error-prone. Automating the process of extracting text information enables greater efficiency and accuracy in data collection and processing.
- **Integration:** Digitized data from HMI screens can be seamlessly integrated into modern software solutions, enabling real-time monitoring, analysis, and reporting, which is vital for decision-making and system optimization.
- **Historical Data Preservation:** For legacy systems, digitizing HMI screen data is essential for preserving historical records. This helps organizations maintain a comprehensive historical database for compliance, troubleshooting, and maintenance purposes.
- **Remote Monitoring:** Digitized data allows for remote monitoring of systems, which is particularly important in industries where physical access to the equipment is limited or risky.

Python code recognizes characters on the screen from an image (e.g. PNG) or video frame (e.g. MP4) using the easyocr library. easyocr is a Python library that simplifies Optical Character Recognition (OCR) tasks. Install easyocr using pip:

In [ ]:
pip install easyocr

Below is an example code snippet to perform text recognition from an image file from a Falcon F5 locator screen that measures ground depth.

<img align=left width=300px src='https://apmonitor.com/dde/uploads/Main/falconf5.png'>

### Import Libraries

In [ ]:
import numpy as np
from PIL import Image
import urllib.request
import easyocr

### Download Image

In [ ]:
# download image
file = 'falconf5.png'
url = 'http://apmonitor.com/dde/uploads/Main/'+file
urllib.request.urlretrieve(url,file)

### Initialize Reader and Extract Values

In [ ]:
# Initialize the PaddleOCR reader
ocr = easyocr.Reader(['en'])

# Read image
img = np.array(Image.open(file))

# Extract values
result = ocr.readtext(img)
for r in result:
    print(f'{r[1]} loc: {r[0]} conf: {np.round(r[2],2)}')

<img align=left width=500px src='https://apmonitor.com/dde/uploads/Main/activity.png'>

### Activity: Digitize Values from Video

Extend the single image OCR to a video file. Record the series of numbers on the screen in a single CSV (Comma Separated Value) file with a video frame identifier. Create a plot of the depth as the locator repeatedly scans to increase accuracy.

```python
video = 'http://apmonitor.com/dde/uploads/Main/dci.mp4'
```

### Import Libraries

In [ ]:
import numpy as np
import urllib.request
import cv2
import easyocr
import matplotlib.pyplot as plt
import pandas as pd

### Download Video

In [ ]:
# download image
file = 'dci.mp4'
url = 'http://apmonitor.com/dde/uploads/Main/'+file
urllib.request.urlretrieve(url,file)

### Read Frames of Video

In [ ]:
# Initialize the reader
ocr = easyocr.Reader(['en'])

# Read video file with OpenCV
cap = cv2.VideoCapture(file)

# results
csv_file = file+'.csv'

# Check if the video file opened
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Loop through each frame in the video
ic=0
while True:
    print(ic)
    ret, frame = cap.read()
    # Break the loop when we reach the end of the video
    if not ret:
        break
    # Convert BGR to RGB
    img = np.array(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    # Identify text
    result = ocr.readtext(img)

    # Write header
    if ic==0:
        fid = open(csv_file,'w')
        # write header
        for i in range(len(result)):
            fid.write(f'data{i},')
        fid.write('frame\n')
        fid.close()

    # write data to file
    fid = open(csv_file,'a')        
    for r in result:
        fid.write(f'{r[1]},')
    fid.write(f'{ic}\n')
    fid.close()

    # increment frame counter
    ic += 1

# Release the video capture object and close file
cap.release()

### Read Data

In [ ]:
# read data
dci = pd.read_csv(csv_file)
# rename columns
dci.columns = ['Read','rm','Depth (ft)','Signal','Frame']
# remove second column
del dci['rm']
# set Frame as index
dci.set_index('Frame',drop=True)
# convert ft' in" to meters
dci['Depth (m)'] = (dci['Depth (ft)'].str.extract("(\d*)'(\d*)\"")
                    .apply(pd.to_numeric, errors='coerce')
                    .mul([0.3048, 0.0254]).sum(axis=1)
                   )
# filter out bad data
dci = dci[dci['Depth (m)']>2]

### Generate Plot

In [ ]:
# create plot of depth
dci['Depth (m)'].plot(figsize=(6,3))
plt.xlabel('Frame'); plt.ylabel('Depth (m)')
plt.tight_layout(); plt.savefig('results.png',dpi=300)
plt.show()